In [20]:
from dotenv import load_dotenv
load_dotenv()

False

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

Indexing (Document Ingestion)

In [4]:
from youtube_transcript_api._api import YouTubeTranscriptApi

video_id = "Gfr50f6ZBvo"

try:
    api = YouTubeTranscriptApi()
    transcript_list = api.fetch(video_id)
    
    transcript = " ".join([snippet.text for snippet in transcript_list])
 
    
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")

Indexing(Text Splitting)

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.create_documents([transcript])

In [6]:
len(chunks)

168

In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [8]:
vector_store = FAISS.from_documents(chunks,embeddings)

In [9]:
vector_store.index_to_docstore_id

{0: 'da612907-323d-449d-ae69-3a34e8c0628c',
 1: '0dab45f4-3be0-4c97-81fe-b4e832fb25fe',
 2: 'b860cf04-9eaf-4d9c-9849-f2435a53bea9',
 3: '3f01f7a3-a62d-4103-8f9e-4119c4819ef5',
 4: '5652da18-e0d4-4f94-8cfd-544f70abfd4a',
 5: '296c103a-c162-4f4a-abb5-c0ab0e7a3782',
 6: '0c968529-3e3b-4aae-a5b1-2687ca40b138',
 7: '7d2c7d41-8f28-4f50-9454-5b8f3263700f',
 8: 'cfc8c49e-f805-4c6a-bc24-403d997d2442',
 9: '39e9c2f7-8df8-49ac-bed2-df0af5259380',
 10: '08d17d68-c361-41f3-a26f-df4c354d7a73',
 11: 'f6252049-9806-4d70-9287-fbd91d91e8c9',
 12: '51bca99f-71cd-46ee-9f27-ec1f6be42ca4',
 13: 'c24e1d45-927c-4104-8bf0-25811a066b0e',
 14: '8dac42a1-2bba-4be6-8262-9e91b48bb52c',
 15: '58ac7c73-4111-48b5-965b-2edc07d87e31',
 16: 'bde942aa-ceb1-4703-97f8-4fd9da991b5e',
 17: '9a07b823-eab1-410d-ba8f-317a6e872749',
 18: '36e08cf2-48c2-43cd-8454-126100a0f0f8',
 19: 'd41b84ae-b562-493f-a214-0e31e88403a6',
 20: '73790564-f47c-4202-a13b-fd6ae1511e51',
 21: 'e9633c07-e82e-4d15-8236-8b862fb51da9',
 22: 'f6b8b1bb-0fa1-

Retrival

In [10]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs = {"k": 4})

In [11]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001CB9FFF4050>, search_kwargs={'k': 4})

In [12]:
result = retriever.invoke("What is deepmind?")
print(result[3].page_content)


ai we would call it now um people like minsky and and and patrick winston and you know all these characters right and used to debate a few of them and they used to think i was mad thinking about that some new advance could be done with learning systems and um i was actually pleased to hear that because at least you know you're on a unique track at that point right even if every all of your you know professors are telling you you're mad that's true and of course in industry uh you can we couldn't get you know as difficult to get two cents together uh and which is hard to imagine now as well given it's the biggest sort of buzzword in in vcs and and fundraising's easy and all these kind of things today so back in 2010 it was very difficult and what we the reason we started then and shane and i used to discuss um uh uh what were the sort of founding tenets of deep mind and it was very various things one was um algorithmic advances so deep learning you know jeff hinton and cohen just had


In [21]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mistral-7B-Instruct-v0.2",
    streaming=False)
model = ChatHuggingFace(llm=llm)

In [22]:
prompt = PromptTemplate(
    template = """
    You are a helpful assistant.
    Answer only from the provided transcript context.
    If the context is insufficient, just say you don't know.
    
    {context}
    Question: {question}
    """,
    input_variables = ["context","question"]
)

In [23]:
question = "is the topic of aliens discussed in this video? if yes then what was discussed?"
retrieved_docs = retriever.invoke(question)

In [24]:
retrieved_docs

[Document(id='9e80db38-4b9a-4b7a-9458-369784c3ea7b', metadata={}, page_content="space age we should have heard a cacophony of voices we should have joined that cacophony of voices and what we did we opened our ears and we heard nothing and many people who argue that there are aliens would say well we haven't really done exhaustive search yet and maybe we're looking in the wrong bands and and we've got the wrong devices and we wouldn't notice what an alien form was like to be so different to what we're used to but you know i'm not i don't really buy that that it shouldn't be as difficult as that like we i think we've searched enough there should be if it were everywhere if it was it should be everywhere we should see dyson's fears being put up sun's blinking in and out you know there should be a lot of evidence for those things and then there are other people argue well the sort of safari view of like well we're a primitive species still because we're not space faring yet and and and we

In [25]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"space age we should have heard a cacophony of voices we should have joined that cacophony of voices and what we did we opened our ears and we heard nothing and many people who argue that there are aliens would say well we haven't really done exhaustive search yet and maybe we're looking in the wrong bands and and we've got the wrong devices and we wouldn't notice what an alien form was like to be so different to what we're used to but you know i'm not i don't really buy that that it shouldn't be as difficult as that like we i think we've searched enough there should be if it were everywhere if it was it should be everywhere we should see dyson's fears being put up sun's blinking in and out you know there should be a lot of evidence for those things and then there are other people argue well the sort of safari view of like well we're a primitive species still because we're not space faring yet and and and we're you know there's some kind of globe like universal rule not to interfere\n\

In [26]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

Generation

In [27]:
answer = model.invoke(final_prompt)
print(answer.content)

HfHubHTTPError: Client error '401 Unauthorized' for url 'https://router.huggingface.co/v1/chat/completions' (Request ID: Root=1-697a873a-2e1210a30b1357a0566981f6;3b68101d-a11a-4247-8303-02eb23ad9c7a)
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/401

Invalid username or password.